In [1]:
from matplotlib import pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual, FileUpload
from IPython.display import display
import pandas as pd
import numpy as np
import ipywidgets as widgets
import scipy.integrate as integrate
import seaborn as sns
import math

import os
import io
import sys 

In [2]:
def gaussian_op(x, mu, τ):
    return np.exp((-np.subtract(x,mu)**2.0)/(2*(τ/2.35)**2))

def parse_filename(df,basename):
    basename_list=basename.replace(" ","_").split("_")
    if basename_list[0] == "Calc":
        df["Dataset"]=" ".join(basename_list[2:])
        df["Kind"]="Simulation"
        df["Temperature"]=basename_list[1]
    else:
        df["Dataset"]=",".join(["experimental",basename_list[1]])
        df["Kind"]="Experiment"
        df["Temperature"]=basename_list[-1]
    df["Filename"]=basename
    df["GDOS (normalized)"]=df["GDOS"]/df["GDOS"].max()
    return df

    
data_path="../data/"
df_list=list()
for filename in os.listdir(data_path):
    basename,extension=os.path.splitext(filename)
    if extension != ".csv":
        continue
    df_tmp=pd.read_csv(os.path.join(data_path,filename),names=["Energy (meV)","GDOS","Error"],skiprows=1)
    df_tmp=parse_filename(df_tmp,basename)
    df_list.append(df_tmp)
df_data=pd.concat(df_list,ignore_index=True)


def apply_blurring(df,A = 0,B = 0,C = 0,T = 0):
    dE = df["Energy (meV)"].diff().mean()
    df["RLT_Broadened_Total"]=df["Energy (meV)"].apply(lambda x : A + B*10**(-11)*T**3*x**2 + C*10**(-5)*x**4 + (-2.46698+(5215.79571/(150*np.sqrt(math.pi/2)))*np.exp(-2*((x-164.07882)/150)**2)))
    df["RLT_Broadened_Boundary"]=df["Energy (meV)"].apply(lambda x : A )
    df["RLT_Broadened_Umklapp"]=df["Energy (meV)"].apply(lambda x : B*10**(-11)*T**3*x**2)
    df["RLT_Broadened_Impurity"]=df["Energy (meV)"].apply(lambda x : C*10**(-5)*x**4)
    df["RLT_Broadened_Instrument"]=df["Energy (meV)"].apply(lambda x : (-2.46698+(5215.79571/(150*np.sqrt(math.pi/2)))*np.exp(-2*((x-164.07882)/150)**2)))
    df_melted=pd.melt(df,id_vars="Energy (meV)",value_vars=["RLT_Broadened_Total","RLT_Broadened_Boundary","RLT_Broadened_Umklapp","RLT_Broadened_Impurity","RLT_Broadened_Instrument"],var_name="Components")
    
    fig, ax = plt.subplots(figsize=(8,5))
    sns.lineplot(data=df_melted
        , x='Energy (meV)'
        , y='value'
        , style='Components'
        , hue='Components'            
     )
    ax.set(title=r'RLT Broadening Function', xlabel=r'Energy (meV)', ylabel=r'Scattering Rate ($s^{-1}$)')

    window_size=np.array(3*df["RLT_Broadened_Total"]/dE).astype(int)
    df.reset_index(drop=True,inplace=True)
    index=df.index
    index_min=df.index.min()
    index_max=df.index.max()
    window_range=np.array([
        np.clip(index-window_size,index_min,index_max),
        np.clip(index+window_size,index_min,index_max)
    ])
    df_size=df.shape[0]
    gdos_window=np.zeros([df_size,df_size])
    energy_window=np.zeros([df_size,df_size])
    for i,(left,right) in enumerate(window_range.T):
        gdos_window[left:right,i]=df["GDOS"].iloc[left:right]
        energy_window[left:right,i]=df["Energy (meV)"].iloc[left:right]
    gdos_blurred=np.sum(gaussian_op(energy_window,df["Energy (meV)"].values,df["RLT_Broadened_Total"].values)*np.divide(gdos_window,df["RLT_Broadened_Total"].values),axis=0)
    df["GDOS (RLT Broadened)"]=gdos_blurred/gdos_blurred.max()
    return df
def plot_with_blurred(df
                      ,experiment_dataset
                      ,experiment_temperature
                      ,simulation_dataset
                      ,simulation_temperature
                      ,RLT_Broadened_Temperature
                      ,Boundary_scattering
                      ,Umklapp_scattering
                      ,Impurity_scattering
                      ,show_original=True):
    df_experiment=df_data.query("Temperature == '%s' and Dataset=='%s'"%(experiment_temperature,experiment_dataset))
    if df_experiment.empty:
        print("Empty set")
        return 
    df_simulation=apply_blurring(df_data.query("Temperature == '%s' and Dataset=='%s' and `Energy (meV)` < %f"%(simulation_temperature,simulation_dataset,df_experiment["Energy (meV)"].max())).copy()
                                 ,A=Boundary_scattering
                                 ,B=Umklapp_scattering
                                 ,C=Impurity_scattering
                                 ,T=RLT_Broadened_Temperature
                                )
    df_simulation.rename(columns={"GDOS (normalized)":"Simulation original","GDOS (RLT Broadened)":"Simulation blurred"},inplace=True)
    df_simulation_melted=pd.melt(df_simulation,id_vars="Energy (meV)",value_vars=["Simulation original","Simulation blurred"],var_name="Dataset",value_name="GDOS (normalized)")

    df_concatenated=pd.concat([df_experiment,df_simulation_melted],ignore_index=True).reset_index(drop=True)
    if not show_original:
        df_concatenated = df_concatenated.query("Dataset != 'Simulation original'")

    fig, ax = plt.subplots(figsize=(8,5))
    sns.lineplot(data=df_concatenated
        , x='Energy (meV)'
        , y='GDOS (normalized)'
        ,hue='Dataset'
     )
    ax.spines['bottom'].set_color('0')
    ax.spines['top'].set_color('1')
    ax.spines['right'].set_color('1')
    ax.spines['left'].set_color('0')
    ax.tick_params(direction='out', width=3, bottom=True, left=True)
    ax.grid(False)
    ax.set_title('Experimental T=%s, Simulation T=%s (%s) and RLT Broadening T=%d K'%(
                      experiment_temperature
                      ,simulation_temperature
                      ,simulation_dataset
                      ,RLT_Broadened_Temperature)
                )
d=interact(plot_with_blurred,df=fixed(df_data.copy())
         ,experiment_dataset=df_data.query("Kind=='Experiment'")["Dataset"].unique()
         ,experiment_temperature=df_data.query("Kind=='Experiment'")["Temperature"].unique()
         ,simulation_dataset=df_data.query("Kind=='Simulation'")["Dataset"].unique()
         ,simulation_temperature=widgets.Dropdown(options=df_data.query("Kind=='Simulation'")["Temperature"].unique(), style = {'description_width': 'initial'})
        ,RLT_Broadened_Temperature=widgets.IntSlider(min=100,max=1000,value=500,step=10, description="RLT Broadened Temperature", style = {'description_width': 'initial'},layout=widgets.Layout(width='500px', height='40px'),continuous_update=False)
        ,Boundary_scattering=widgets.FloatSlider(min=0, max=10, value=0, step=0.01, description="Boundary Scattering", style = {'description_width': 'initial'}, layout=widgets.Layout(width='500px', height='40px'))
        ,Umklapp_scattering=widgets.FloatSlider(min=0, max=10, value=0, step=0.01, description="Umklapp Scattering", style = {'description_width': 'initial'}, layout=widgets.Layout(width='500px', height='40px'))
        ,Impurity_scattering=widgets.FloatSlider(min=0, max=10, value=0, step=0.01, description="Impurity Scattering", style = {'description_width': 'initial'}, layout=widgets.Layout(width='500px', height='40px'))
         )

interactive(children=(Dropdown(description='experiment_dataset', options=('experimental,Feb', 'experimental,Ja…

In [3]:
def lattice_thermal_conductivity(df,l = 7.638*10**-12,v = 1,θ = 1):
#l=Boltzmann constant / Reduced Planck constant (s/K)
#v=Group velocity (m/s)
#θ=Debeye Temperature (K)
#T = Temp - create list of temps to use? Not sure?
#Relaxation time broadening function = calculated as a function of E and T - previously selected with slider
    df["Lattice_Thermal_Conductivity"]=df["Temperature (T)"].apply(lambda x : ((1.38*10**-23/2*np.pi**2*v)*(l*T)**3*(l*x/T)**4*np.exp(l*x/T)/(df["RLT_Broadened_Total"]*(np.exp(l*x/T)-1)**2)))
    df["Lattice_Thermal_Conductivity_Boundary"]=df["Temperature (T)"].apply(lambda x : ((1.38*10**-23/2*np.pi**2*v)*(l*T)**3*(l*x/T)**4*np.exp(l*x/T)/(df["RLT_Broadened_Boundary"]*(np.exp(l*x/T)-1)**2)))
    df["Lattice_Thermal_Conductivity_Umklapp"]=df["Temperature (T)"].apply(lambda x : ((1.38*10**-23/2*np.pi**2*v)*(l*T)**3*(l*x/T)**4*np.exp(l*x/T)/(df["RLT_Broadened_Umklapp"]*(np.exp(l*x/T)-1)**2)))
    df["Lattice_Thermal_Conductivity_Impurity"]=df["Temperature (T)"].apply(lambda x : ((1.38*10**-23/2*np.pi**2*v)*(l*T)**3*(l*x/T)**4*np.exp(l*x/T)/(df["RLT_Broadened_Impurity"]*(np.exp(l*x/T)-1)**2)))
    df_melted_cond=pd.melt(df,id_vars="Energy (meV)",value_vars=["Lattice_Thermal_Conductivity", "Lattice_Thermal_Conductivity_Boundary","Lattice_Thermal_Conductivity_Umklapp","Lattice_Thermal_Conductivity_Impurity"],var_name="Components")
   
    #I = integrate.quad(df["lat_therm_cond"], 0, θ/T)

    #print(df["Lattice_Thermal_Conductivity"])

                        # ,Debeye_Temperature
                     # ,Group_Velocity
                       #          ,θ=Debeye_Temperature
                       #          ,v=Group_Velocity
          # ,Debeye_Temperature=widgets.FloatSlider(min=0, max=500, value=0, step=0.01, description="Impurity Scattering", style = {'description_width': 'initial'}, layout=widgets.Layout(width='500px', height='40px'))  
       # ,Group_Velocity=widgets.FloatSlider(min=0, max=5000, value=0, step=0.01, description="Impurity Scattering", style = {'description_width': 'initial'}, layout=widgets.Layout(width='500px', height='40px'))
   #THE THERMAL CONDUCTIVITY NEEDS TO HAVE A DEFINED TEMPERATURE FOR THE CALCULATION OF ENERGY AND THE SCATTERING RATE
#IT ALSO NEEDS TO BE TEMPERATURE DEPENDENT AS TO PLOT THE THERMAL CONDUCTIVITY AGAINST TEMPERATURE
#SO THE INTEGRATION OVER ENERGY NEEDS TO COME FIRST WITH A DEFINITE TEMPERATURE
#THEN PLOT THIS CALCULATED VALUE WITH THE TEMPERATURE DEPENDENCE 

    fig, ax = plt.subplots(figsize=(8,5))
    sns.lineplot(data=df_melt_cond
        , x='Energy (meV)'
        , y='value'
        , style='Components'
        , hue='Components'            
     )
    ax.set(title=r'Lattice Thermal Conductivity', xlabel=r'Temperature (K)', ylabel=r'κ')

   

In [9]:

def read_uploaded_file(file_data):
    df=pd.read_csv(io.BytesIO(file_data["content"]),names=["Energy (meV)","GDOS","Error"],skiprows=1)
    basename,extension=os.path.splitext(file_data["name"])
    df=parse_filename(df,basename)
    return df


def apply_RLT_Broadening(df,A = 0,B = 0,C = 0,T = 0, v = 1,θ = 1):
    dE = df["Energy (meV)"].diff().mean()
    df["RLT_Broadened_Total"]=df["Energy (meV)"].apply(lambda x : A + B*10**(-11)*T**3*x**2 + C*10**(-5)*x**4 + (-2.46698+(5215.79571/(150*np.sqrt(math.pi/2)))*np.exp(-2*((x-164.07882)/150)**2)))
    df["RLT_Broadened_Boundary"]=df["Energy (meV)"].apply(lambda x : A )
    df["RLT_Broadened_Umklapp"]=df["Energy (meV)"].apply(lambda x : B*10**(-11)*T**3*x**2)
    df["RLT_Broadened_Impurity"]=df["Energy (meV)"].apply(lambda x : C*10**(-5)*x**4)
    df["RLT_Broadened_Instrument"]=df["Energy (meV)"].apply(lambda x : (-2.46698+(5215.79571/(150*np.sqrt(math.pi/2)))*np.exp(-2*((x-164.07882)/150)**2)))
    df_melted=pd.melt(df,id_vars="Energy (meV)",value_vars=["RLT_Broadened_Total","RLT_Broadened_Boundary","RLT_Broadened_Umklapp","RLT_Broadened_Impurity","RLT_Broadened_Instrument"],var_name="Components")
    
    fig, ax = plt.subplots(figsize=(8,5))
    sns.lineplot(data=df_melted
        , x='Energy (meV)'
        , y='value'
        , style='Components'
        , hue='Components'            
     )
    ax.set(title=r'RLT Broadening Function', xlabel=r'Energy (meV)', ylabel=r'Scattering Rate ($s^{-1}$)')

    
    l = 7.638*10**-12
    const=(1.38*10**-23)/(2*np.pi**2*v)*(l*T)**3
    df["kappa_L"]=df["Energy (meV)"].apply(lambda y : const*integrate.quad(thermal_conductivity_integral, 0, θ/T,args=(tau_c_inv(y,A,B,C)))[0])
        
    fig, ax = plt.subplots(figsize=(8,5))
    sns.lineplot(data=df
        , x='Energy (meV)'
        , y='kappa_L'
     )



    
    window_size=np.array(3*df["RLT_Broadened_Total"]/dE).astype(int)
    df.reset_index(drop=True,inplace=True)
    index=df.index
    index_min=df.index.min()
    index_max=df.index.max()
    window_range=np.array([
        np.clip(index-window_size,index_min,index_max),
        np.clip(index+window_size,index_min,index_max)
    ])
    df_size=df.shape[0]
    gdos_window=np.zeros([df_size,df_size])
    energy_window=np.zeros([df_size,df_size])
    for i,(left,right) in enumerate(window_range.T):
        gdos_window[left:right,i]=df["GDOS"].iloc[left:right]
        energy_window[left:right,i]=df["Energy (meV)"].iloc[left:right]
    gdos_blurred=np.sum(gaussian_op(energy_window,df["Energy (meV)"].values,df["RLT_Broadened_Total"].values)*np.divide(gdos_window,df["RLT_Broadened_Total"].values),axis=0)
    df["GDOS (RLT Broadened)"]=gdos_blurred/gdos_blurred.max()
    return df

def dict2list(data_dict):
    data_list=list()
    for k,v in data_dict.items():
        data.append({"name":k,"content":v["content"]})
    return data_list

def tau_c_inv(x, A = 0,B = 0,C = 0,T = 0):
    return A + B*10**(-11)*T**3*x**2 + C*10**(-5)*x**4 + (-2.46698+(5215.79571/(150*np.sqrt(math.pi/2)))*np.exp(-2*((x-164.07882)/150)**2))

def thermal_conductivity_integral(x,tau_c_inv):
    return x**4*np.exp(x)/(tau_c_inv*(np.exp(x)-1)**2)




def thermal_conductivity(reference
                         ,to_process
                      ,RLT_Broadened_Temperature
                      ,Boundary_scattering
                      ,Umklapp_scattering
                      ,Impurity_scattering
                      ,Debeye_Temperature
                      ,Group_Velocity
                      ,show_original=True):
    
    if len(reference.value) == 0 or len(to_process.value) == 0:
        print("Upload data first")
        return 
    if type(reference.value) == tuple:
        data=reference.value
    else:
        data=dict2list(reference.value)
    
    df_experiment=read_uploaded_file(data[0])
    if df_experiment.empty:
        print("Empty set")
        return

    if type(to_process.value) == tuple:
        data=to_process.value
    else:
        data=dict2list(to_process.value)

    df_data=read_uploaded_file(data[0])
    if df_data.empty:
        print("Empty set")
        return    
    df_simulation=apply_RLT_Broadening(df_data.query("`Energy (meV)` < %f"%(df_experiment["Energy (meV)"].max())).copy()
                                 ,A=Boundary_scattering
                                 ,B=Umklapp_scattering
                                 ,C=Impurity_scattering
                                 ,T=RLT_Broadened_Temperature
                                 ,v = Group_Velocity
                                 ,θ = Debeye_Temperature
                                )
    filename=df_data["Filename"].unique()[0]
    df_simulation.rename(columns={"GDOS (normalized)":"%s (original)"%filename, "GDOS (RLT Broadened)":"%s (RLT Broadened)"%filename},inplace=True)
    df_simulation_melted=pd.melt(df_simulation,id_vars="Energy (meV)",value_vars=["%s (original)"%filename,"%s (RLT Broadened)"%filename],var_name="Filename",value_name="GDOS (normalized)")

    df_concatenated=pd.concat([df_experiment,df_simulation_melted],ignore_index=True).reset_index(drop=True)
    if not show_original:
        df_concatenated = df_concatenated.query("Dataset != 'Simulation original'")

    fig, ax = plt.subplots(figsize=(8,5))
    sns.lineplot(data=df_concatenated
        , x='Energy (meV)'
        , y='GDOS (normalized)'
        ,hue='Filename'
     )
    ax.spines['bottom'].set_color('0')
    ax.spines['top'].set_color('1')
    ax.spines['right'].set_color('1')
    ax.spines['left'].set_color('0')
    ax.tick_params(direction='out', width=3, bottom=True, left=True)
    ax.grid(False)
    ax.legend(loc='lower center', bbox_to_anchor=(0.5, 1.1),ncol=3)
    ax.set_title('Reference T=%s, Data T=%s and RLT Broadening T=%d K'%(
                      df_experiment["Temperature"].unique()[0]
                      ,df_simulation["Temperature"].unique()[0]
                      ,RLT_Broadened_Temperature)
                )
    
reference = FileUpload(accept='.csv',multiple=False,description="Reference")
to_process = FileUpload(accept='.csv',multiple=False,description="To process")
display(reference)
display(to_process)
d=interact_manual(thermal_conductivity
                  ,reference=fixed(reference)
                  ,to_process=fixed(to_process)
       ,RLT_Broadened_Temperature=widgets.IntSlider(min=100,max=1000,value=500,step=10, description="RLT Broadened Temperature", style = {'description_width': 'initial'},layout=widgets.Layout(width='500px', height='40px'),continuous_update=False)
        ,Boundary_scattering=widgets.FloatSlider(min=0, max=10, value=0, step=0.01, description="Boundary Scattering", style = {'description_width': 'initial'}, layout=widgets.Layout(width='500px', height='40px'))
        ,Umklapp_scattering=widgets.FloatSlider(min=0, max=10, value=0, step=0.01, description="Umklapp Scattering", style = {'description_width': 'initial'}, layout=widgets.Layout(width='500px', height='40px'))
        ,Impurity_scattering=widgets.FloatSlider(min=0, max=10, value=0, step=0.01, description="Impurity Scattering", style = {'description_width': 'initial'}, layout=widgets.Layout(width='500px', height='40px'))
        ,Debeye_Temperature=widgets.FloatSlider(min=100, max=500, value=0, step=0.01, description="Debeye Temperature", style = {'description_width': 'initial'}, layout=widgets.Layout(width='500px', height='40px'))  
        ,Group_Velocity=widgets.FloatSlider(min=1, max=5000, value=0, step=0.01, description="Group Velocity", style = {'description_width': 'initial'}, layout=widgets.Layout(width='500px', height='40px'))
         )

FileUpload(value=(), accept='.csv', description='Reference')

FileUpload(value=(), accept='.csv', description='To process')

interactive(children=(IntSlider(value=500, continuous_update=False, description='RLT Broadened Temperature', l…